# Лабораторная работа №2: Логистическая и линейная регрессия

1. Бейзлайн sklearn
2. Улучшение через подбор гиперпараметров
3. Собственная имплементация

## Импорт библиотек
Импортируем необходимые библиотеки для работы с данными, моделями машинного обучения и метриками качества.

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import f1_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import openml
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')
print("Импорт завершён")

Импорт завершён


### Датасет классификации: APS Failure at Scania Trucks
Загружаем данные из OpenML (ID 41138). Заполняем пропуски медианой и сэмплируем 15000 объектов для ускорения экспериментов. Разделяем на train/test с сохранением стратификации.

In [13]:
# Загрузка классификации
dataset = openml.datasets.get_dataset(41138)
X_clf, y_clf, _, _ = dataset.get_data(target=dataset.default_target_attribute)
y_clf_enc = (y_clf == 'pos').astype(int)
imputer = SimpleImputer(strategy='median')
X_clf_imp = pd.DataFrame(imputer.fit_transform(X_clf), columns=X_clf.columns)
np.random.seed(42)
idx = np.random.choice(len(X_clf_imp), 15000, replace=False)
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf_imp.iloc[idx], y_clf_enc.iloc[idx], test_size=0.2, random_state=42, stratify=y_clf_enc.iloc[idx]
)
print(f"Классификация: {X_clf_train.shape}")

Классификация: (12000, 170)


### Датасет регрессии: Avocado Prices
Загружаем данные о ценах авокадо с Kaggle. Кодируем категориальные признаки (тип, регион) и выбираем числовые признаки для предсказания средней цены.

In [14]:
# Загрузка регрессии: Avocado Prices
path = kagglehub.dataset_download("neuromusic/avocado-prices")
df = pd.read_csv(os.path.join(path, "avocado.csv"))
df['type_enc'] = LabelEncoder().fit_transform(df['type'])
df['region_enc'] = LabelEncoder().fit_transform(df['region'])
features = ['Total Volume', '4046', '4225', '4770', 'Total Bags', 'year', 'type_enc', 'region_enc']
X_reg = df[features].values
y_reg = df['AveragePrice'].values
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
print(f"Регрессия: {X_reg_train.shape}")

Регрессия: (14599, 8)


## 2. Бейзлайн

Масштабируем признаки с помощью StandardScaler и обучаем базовую логистическую регрессию. Масштабирование важно для корректной работы регуляризации и ускорения сходимости.

In [15]:
# Масштабирование
scaler_clf = StandardScaler()
X_clf_train_sc = scaler_clf.fit_transform(X_clf_train)
X_clf_test_sc = scaler_clf.transform(X_clf_test)
scaler_reg = StandardScaler()
X_reg_train_sc = scaler_reg.fit_transform(X_reg_train)
X_reg_test_sc = scaler_reg.transform(X_reg_test)

# Бейзлайн классификация
log_reg_base = LogisticRegression(max_iter=1000)
log_reg_base.fit(X_clf_train_sc, y_clf_train)
y_pred_base = log_reg_base.predict(X_clf_test_sc)
y_proba_base = log_reg_base.predict_proba(X_clf_test_sc)[:, 1]
f1_base = f1_score(y_clf_test, y_pred_base)
roc_base = roc_auc_score(y_clf_test, y_proba_base)
print(f"=== БЕЙЗЛАЙН: Logistic Regression ===")
print(f"F1: {f1_base:.4f}, ROC-AUC: {roc_base:.4f}")

=== БЕЙЗЛАЙН: Logistic Regression ===
F1: 0.6833, ROC-AUC: 0.9356


Обучаем базовую модель LinearRegression из sklearn. Оцениваем качество по RMSE и R². Линейная регрессия не требует подбора гиперпараметров.

In [16]:
# Бейзлайн регрессия
lin_reg_base = LinearRegression()
lin_reg_base.fit(X_reg_train_sc, y_reg_train)
y_pred_reg_base = lin_reg_base.predict(X_reg_test_sc)
rmse_base = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_base))
r2_base = r2_score(y_reg_test, y_pred_reg_base)
print(f"=== БЕЙЗЛАЙН: Linear Regression ===")
print(f"RMSE: {rmse_base:.4f}, R²: {r2_base:.4f}")

=== БЕЙЗЛАЙН: Linear Regression ===
RMSE: 0.3133, R²: 0.3892


## 3. Улучшение бейзлайна

Подбор гиперпараметров логистической регрессии: коэффициент регуляризации `C` и балансировка классов `class_weight`. Оптимизируем по F1-score с помощью кросс-валидации.

In [17]:
# GridSearch для классификации
param_grid = {'C': [0.1, 1.0, 10.0], 'class_weight': [None, 'balanced']}
grid_clf = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_clf.fit(X_clf_train_sc, y_clf_train)

print(f"Лучшие параметры: {grid_clf.best_params_}")
y_pred_imp = grid_clf.predict(X_clf_test_sc)
y_proba_imp = grid_clf.predict_proba(X_clf_test_sc)[:, 1]
f1_imp = f1_score(y_clf_test, y_pred_imp)
roc_imp = roc_auc_score(y_clf_test, y_proba_imp)
print(f"=== УЛУЧШЕННЫЙ ===")
print(f"F1: {f1_imp:.4f} ({f1_imp-f1_base:+.4f})")
print(f"ROC-AUC: {roc_imp:.4f}")

Лучшие параметры: {'C': 1.0, 'class_weight': None}
=== УЛУЧШЕННЫЙ ===
F1: 0.6833 (+0.0000)
ROC-AUC: 0.9356


Применяем Ridge-регрессию (L2-регуляризация) для борьбы с переобучением. Подбираем коэффициент регуляризации `alpha` через GridSearchCV.

In [18]:
# Ridge регрессия
grid_reg = GridSearchCV(Ridge(), {'alpha': [0.1, 1.0, 10.0, 100.0]}, cv=3, scoring='r2', n_jobs=-1)
grid_reg.fit(X_reg_train_sc, y_reg_train)
print(f"Лучший alpha: {grid_reg.best_params_['alpha']}")
y_pred_reg_imp = grid_reg.predict(X_reg_test_sc)
rmse_imp = np.sqrt(mean_squared_error(y_reg_test, y_pred_reg_imp))
r2_imp = r2_score(y_reg_test, y_pred_reg_imp)
print(f"=== УЛУЧШЕННЫЙ: Ridge ===")
print(f"RMSE: {rmse_imp:.4f} ({rmse_imp-rmse_base:+.4f})")
print(f"R²: {r2_imp:.4f} ({r2_imp-r2_base:+.4f})")

Лучший alpha: 10.0
=== УЛУЧШЕННЫЙ: Ridge ===
RMSE: 0.3133 (+0.0000)
R²: 0.3890 (-0.0002)


## 4. Собственная имплементация

Реализуем алгоритмы с нуля:
- **MyLinearRegression**: Аналитическое решение через псевдообратную матрицу (формула нормального уравнения)
- **MyLogisticRegression**: Градиентный спуск с L2-регуляризацией. Используем сигмоиду для преобразования в вероятности

In [19]:
class MyLinearRegression:
    """Линейная регрессия (аналитическое решение)"""
    def __init__(self):
        self.w = None
        self.b = None
    
    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        X_b = np.c_[np.ones(len(X)), X]
        theta = np.linalg.pinv(X_b.T @ X_b) @ X_b.T @ y
        self.b, self.w = theta[0], theta[1:]
        return self
    
    def predict(self, X):
        return np.array(X) @ self.w + self.b

class MyLogisticRegression:
    """Логистическая регрессия (градиентный спуск)"""
    def __init__(self, lr=0.1, n_iter=1000, C=1.0):
        self.lr, self.n_iter, self.C = lr, n_iter, C
        self.w = self.b = None
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        n, m = X.shape
        self.w, self.b = np.zeros(m), 0
        for _ in range(self.n_iter):
            p = self._sigmoid(X @ self.w + self.b)
            error = p - y
            self.w -= self.lr * ((X.T @ error) / n + self.w / self.C)
            self.b -= self.lr * np.mean(error)
        return self
    
    def predict_proba(self, X):
        p = self._sigmoid(np.array(X) @ self.w + self.b)
        return np.c_[1-p, p]
    
    def predict(self, X):
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(int)

print("Реализации готовы!")

Реализации готовы!


Тестируем собственную реализацию логистической регрессии. Обучаем на масштабированных данных и сравниваем результаты со sklearn-бейзлайном по метрикам F1 и ROC-AUC.

In [20]:
# Своя классификация
my_log = MyLogisticRegression(lr=0.1, n_iter=2000, C=1.0)
my_log.fit(X_clf_train_sc, y_clf_train.values)
my_pred = my_log.predict(X_clf_test_sc)
my_proba = my_log.predict_proba(X_clf_test_sc)[:, 1]
my_f1 = f1_score(y_clf_test, my_pred)
my_roc = roc_auc_score(y_clf_test, my_proba)
print(f"=== СВОЯ: Логистическая регрессия ===")
print(f"F1: {my_f1:.4f} (sklearn: {f1_base:.4f})")
print(f"ROC-AUC: {my_roc:.4f}")

=== СВОЯ: Логистическая регрессия ===
F1: 0.3571 (sklearn: 0.6833)
ROC-AUC: 0.9726


Тестируем собственную реализацию линейной регрессии на задаче предсказания цены авокадо. Используем аналитическое решение (метод наименьших квадратов). Результаты должны совпадать со sklearn.

In [21]:
# Своя регрессия
my_lin = MyLinearRegression()
my_lin.fit(X_reg_train_sc, y_reg_train)
my_pred_reg = my_lin.predict(X_reg_test_sc)
my_rmse = np.sqrt(mean_squared_error(y_reg_test, my_pred_reg))
my_r2 = r2_score(y_reg_test, my_pred_reg)
print(f"=== СВОЯ: Линейная регрессия ===")
print(f"RMSE: {my_rmse:.4f} (sklearn: {rmse_base:.4f})")
print(f"R²: {my_r2:.4f} (sklearn: {r2_base:.4f})")

=== СВОЯ: Линейная регрессия ===
RMSE: 0.3133 (sklearn: 0.3133)
R²: 0.3892 (sklearn: 0.3892)


## Итоговая сводка

Сравниваем все модели: бейзлайн sklearn, улучшенный вариант и собственную реализацию. Для классификации — F1 и ROC-AUC, для регрессии — RMSE и R².

In [22]:
print("="*70)
print("ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №2 (Линейные модели)")
print("="*70)
print(f"\n{'КЛАССИФИКАЦИЯ':-^70}")
print(f"{'Модель':<30} {'F1':<12} {'ROC-AUC':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {f1_base:<12.4f} {roc_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {f1_imp:<12.4f} {roc_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_f1:<12.4f} {my_roc:<12.4f}")
print(f"\n{'РЕГРЕССИЯ (Avocado Prices)':-^70}")
print(f"{'Модель':<30} {'RMSE':<12} {'R²':<12}")
print("-"*54)
print(f"{'Бейзлайн sklearn':<30} {rmse_base:<12.4f} {r2_base:<12.4f}")
print(f"{'Улучшенный sklearn':<30} {rmse_imp:<12.4f} {r2_imp:<12.4f}")
print(f"{'Своя реализация':<30} {my_rmse:<12.4f} {my_r2:<12.4f}")

ИТОГОВАЯ СВОДКА: ЛАБОРАТОРНАЯ РАБОТА №2 (Линейные модели)

----------------------------КЛАССИФИКАЦИЯ-----------------------------
Модель                         F1           ROC-AUC     
------------------------------------------------------
Бейзлайн sklearn               0.6833       0.9356      
Улучшенный sklearn             0.6833       0.9356      
Своя реализация                0.3571       0.9726      

----------------------РЕГРЕССИЯ (Avocado Prices)----------------------
Модель                         RMSE         R²          
------------------------------------------------------
Бейзлайн sklearn               0.3133       0.3892      
Улучшенный sklearn             0.3133       0.3890      
Своя реализация                0.3133       0.3892      
